In [34]:
import json
from itertools import product
from pathlib import Path

from evals.data import get_jsonl

##### Create sets of samples by combining winelists and set of criteria with a cartesian product.

In [40]:
cwd = Path.cwd()

winelist_relative_path = Path("registry/data/sommelia/winelists.jsonl")
criteria_relative_path = Path("registry/data/sommelia/criteria.jsonl")

winelist_path = cwd / winelist_relative_path
criteria_path = cwd / criteria_relative_path

In [41]:
winelists = get_jsonl(winelist_path)
criteria = get_jsonl(criteria_path)

In [46]:
# Cartesian product of winelists and criteria
samples = [{**winelist, **criterion} for winelist, criterion in product(winelists, criteria)]

In [44]:
def write_to_jsonl(data, file_name:str):
    path = cwd / Path(f"registry/data/sommelia/{file_name}.jsonl")
    with open(path, "a") as f:
        for line in data:
            json.dump(line, f, ensure_ascii=False)
            f.write("\n")

In [45]:
write_to_jsonl(samples, "samples")